<a href="https://colab.research.google.com/github/AliciaAPerez/Death_Analytics_using_CDC_Data_with_Machine_Learning/blob/Claude/Death_stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-04-10 19:16:19--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2021-04-10 19:16:19 (6.00 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2015_data.csv"
spark.sparkContext.addFile(url)
death_2015 = spark.read.csv(SparkFiles.get("2015_data.csv"), sep=",", header=True)

# Show DataFrame
death_2015.show()

+---------------+-----------------------+-----------------------+------------------------+--------------+---+---------------+----------+---------------------+-------------+-------------+-------------+--------------------+-----------------------------------+--------------+--------------------+-----------------+--------------+---------------+---------------------+-------+-------------+------------------------------------------------------+----------------------+----------------+----------------+-----------------------+---------------+--------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------------------+-----

In [5]:
#removing unneded columns
death_2015_clean = death_2015.drop("education_1989_revision","education_reporting_flag","age_substitution_flag","age_recode_52","age_recode_27","age_recode_12", "infant_age_recode_22",
"place_of_death_and_decedents_status","method_of_disposition","autopsy", "activity_code", "place_of_injury_for_causes_w00_y34_except_y06_and_y07_", "358_cause_recode", "113_cause_recode",
"130_infant_cause_recode", "39_cause_recode", "bridged_race_flag", "race_imputation_flag", "race_recode_3", "race_recode_5", "hispanic_origin")

In [6]:
death_2015_clean.show()

+---------------+-----------------------+--------------+---+---------------+----------+--------------+--------------------+-----------------+--------------+---------------+----------------------+--------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-----------------

In [7]:
#removing any individual under the age of one due to how their recorded in the data per the documentation
#for any subject where "detail_age_type" = 1, the detail_age is the age in years, but "detail_age_type" = 2 through 9, age is months(2), days(4), hours(5), minutes(6), or not stated (9)

death_2015_clean_adults = death_2015_clean.filter((death_2015_clean.detail_age_type == 1))
death_2015_clean_adults.show()

+---------------+-----------------------+--------------+---+---------------+----------+--------------+--------------------+-----------------+--------------+---------------+----------------------+--------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-----------------

In [8]:
#removed anyone under the age of 1 (not in category 1), now removing the "detail_age_type" category as it will no longer be needed
death_2015_adults = death_2015_clean_adults.drop("detail_age_type")
death_2015_adults.show()

+---------------+-----------------------+--------------+---+----------+--------------+--------------------+-----------------+--------------+---------------+----------------------+--------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------

In [9]:
death_2015_adults.printSchema()

root
 |-- resident_status: string (nullable = true)
 |-- education_2003_revision: string (nullable = true)
 |-- month_of_death: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- detail_age: string (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- day_of_week_of_death: string (nullable = true)
 |-- current_data_year: string (nullable = true)
 |-- injury_at_work: string (nullable = true)
 |-- manner_of_death: string (nullable = true)
 |-- icd_code_10th_revision: string (nullable = true)
 |-- number_of_entity_axis_conditions: string (nullable = true)
 |-- entity_condition_1: string (nullable = true)
 |-- entity_condition_2: string (nullable = true)
 |-- entity_condition_3: string (nullable = true)
 |-- entity_condition_4: string (nullable = true)
 |-- entity_condition_5: string (nullable = true)
 |-- entity_condition_6: string (nullable = true)
 |-- entity_condition_7: string (nullable = true)
 |-- entity_condition_8: string (nullable = true)
 |-- entity_

In [10]:
#row count
death_2015_adults.count()

2694400

In [11]:
#need to get some data on the figures for the condition columns but they are strings
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType
death_2015_update = death_2015_adults.withColumn("number_of_entity_axis_conditions",col("number_of_entity_axis_conditions").cast("integer")).withColumn("number_of_record_axis_conditions",col("number_of_record_axis_conditions").cast("integer"))
death_2015_update.printSchema()

root
 |-- resident_status: string (nullable = true)
 |-- education_2003_revision: string (nullable = true)
 |-- month_of_death: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- detail_age: string (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- day_of_week_of_death: string (nullable = true)
 |-- current_data_year: string (nullable = true)
 |-- injury_at_work: string (nullable = true)
 |-- manner_of_death: string (nullable = true)
 |-- icd_code_10th_revision: string (nullable = true)
 |-- number_of_entity_axis_conditions: integer (nullable = true)
 |-- entity_condition_1: string (nullable = true)
 |-- entity_condition_2: string (nullable = true)
 |-- entity_condition_3: string (nullable = true)
 |-- entity_condition_4: string (nullable = true)
 |-- entity_condition_5: string (nullable = true)
 |-- entity_condition_6: string (nullable = true)
 |-- entity_condition_7: string (nullable = true)
 |-- entity_condition_8: string (nullable = true)
 |-- entity

In [12]:
#need to know on average how many underlying conditions there are 20 options are available
death_2015_update.select("number_of_entity_axis_conditions", "number_of_record_axis_conditions").describe().show()

+-------+--------------------------------+--------------------------------+
|summary|number_of_entity_axis_conditions|number_of_record_axis_conditions|
+-------+--------------------------------+--------------------------------+
|  count|                         2694400|                         2694400|
|   mean|               3.103408551068884|               2.953636059976247|
| stddev|              1.9167837941567076|              1.7928775740469587|
|    min|                               1|                               1|
|    max|                              15|                              15|
+-------+--------------------------------+--------------------------------+



In [13]:
#per documentation of the data, record_axis will be removed as it is for the death certificate, and the entity is pre-known conditions
#for the entity conditions, droppping all but the average of 3, and removing the number of entity as it is no longer needed 
death_2015_cleaned = death_2015_update.drop("number_of_entity_axis_conditions","entity_condition_4","entity_condition_5","entity_condition_6","entity_condition_7","entity_condition_8",
                                            "entity_condition_9","entity_condition_10","entity_condition_11","entity_condition_12","entity_condition_13","entity_condition_14",
                                            "entity_condition_15","entity_condition_16","entity_condition_17","entity_condition_18","entity_condition_19","entity_condition_20",
                                            "number_of_record_axis_conditions", "record_condition_1", "record_condition_2","record_condition_3","record_condition_4","record_condition_5",
                                            "record_condition_6","record_condition_7","record_condition_8","record_condition_9","record_condition_10","record_condition_11","record_condition_12",
                                            "record_condition_13","record_condition_14","record_condition_15","record_condition_16","record_condition_17","record_condition_18","record_condition_19",
                                            "record_condition_20",)

In [14]:
death_2015_cleaned.show()

+---------------+-----------------------+--------------+---+----------+--------------+--------------------+-----------------+--------------+---------------+----------------------+------------------+------------------+------------------+----+--------------------------+
|resident_status|education_2003_revision|month_of_death|sex|detail_age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|icd_code_10th_revision|entity_condition_1|entity_condition_2|entity_condition_3|race|hispanic_originrace_recode|
+---------------+-----------------------+--------------+---+----------+--------------+--------------------+-----------------+--------------+---------------+----------------------+------------------+------------------+------------------+----+--------------------------+
|              1|                      3|            01|  M|       084|             M|                   1|             2015|             U|              7|                  I500|            11

In [15]:
#list of columns
death_2015_cleaned.schema.names

['resident_status',
 'education_2003_revision',
 'month_of_death',
 'sex',
 'detail_age',
 'marital_status',
 'day_of_week_of_death',
 'current_data_year',
 'injury_at_work',
 'manner_of_death',
 'icd_code_10th_revision',
 'entity_condition_1',
 'entity_condition_2',
 'entity_condition_3',
 'race',
 'hispanic_originrace_recode']

In [16]:
#cleaning up names
death_2015_columns = death_2015_cleaned.withColumnRenamed("resident_status","resident").withColumnRenamed("education_2003_revision","education").withColumnRenamed("detail_age", "age").withColumnRenamed("current_date_year", "year").withColumnRenamed("icd_code_10th_revision", "icd_death_code").withColumnRenamed("hispanic_originrace_recode", "hispanic")
death_2015_columns.show()

+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|icd_death_code|entity_condition_1|entity_condition_2|entity_condition_3|race|hispanic|
+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|       1|        3|            01|  M|084|             M|                   1|             2015|             U|              7|          I500|            11I500|            61L031|              null|  01|       6|
|       1|        6|            01|  M|070|             M|                   2|             2015|             U|              7|          C3

In [17]:
#what are the types
death_2015_columns.printSchema()

root
 |-- resident: string (nullable = true)
 |-- education: string (nullable = true)
 |-- month_of_death: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- day_of_week_of_death: string (nullable = true)
 |-- current_data_year: string (nullable = true)
 |-- injury_at_work: string (nullable = true)
 |-- manner_of_death: string (nullable = true)
 |-- icd_death_code: string (nullable = true)
 |-- entity_condition_1: string (nullable = true)
 |-- entity_condition_2: string (nullable = true)
 |-- entity_condition_3: string (nullable = true)
 |-- race: string (nullable = true)
 |-- hispanic: string (nullable = true)



In [18]:
#making the integer columns integer, the death and entity conditions cannot be integers as they have letters as well as numbers
death_2015_int = death_2015_columns.withColumn("resident",col("resident").cast("integer")).withColumn("education",col("education").cast("integer")).withColumn("month_of_death",col("month_of_death").cast("integer")).withColumn("age",col("age").cast("integer")).withColumn("day_of_week_of_death",col("day_of_week_of_death").cast("integer")).withColumn("current_data_year",col("current_data_year").cast("integer")).withColumn("manner_of_death",col("manner_of_death").cast("integer")).withColumn("race",col("race").cast("integer")).withColumn("hispanic",col("hispanic").cast("integer"))
death_2015_int.show()

+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|icd_death_code|entity_condition_1|entity_condition_2|entity_condition_3|race|hispanic|
+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|       1|        3|             1|  M| 84|             M|                   1|             2015|             U|              7|          I500|            11I500|            61L031|              null|   1|       6|
|       1|        6|             1|  M| 70|             M|                   2|             2015|             U|              7|          C3

In [19]:
#filling in the null values with zero
#we want to maintain the rows, but they will not have other conditions or less than 3
death_2015_final = death_2015_int.fillna(value="0",subset=["entity_condition_1"]).fillna(value="0",subset=["entity_condition_2"]).fillna(value="0",subset=["entity_condition_3"])
death_2015_final.show()

+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|icd_death_code|entity_condition_1|entity_condition_2|entity_condition_3|race|hispanic|
+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|       1|        3|             1|  M| 84|             M|                   1|             2015|             U|              7|          I500|            11I500|            61L031|                 0|   1|       6|
|       1|        6|             1|  M| 70|             M|                   2|             2015|             U|              7|          C3

In [20]:
#function for other data sets to be cleaned up - only can be used for 2014 and 2015 and newer, older pages use a different education marker
def data_clean_up(data):
  data = data.drop("education_1989_revision","education_reporting_flag","age_substitution_flag","age_recode_52","age_recode_27","age_recode_12", "infant_age_recode_22",
"place_of_death_and_decedents_status","method_of_disposition","autopsy", "activity_code", "place_of_injury_for_causes_w00_y34_except_y06_and_y07_", "358_cause_recode", "113_cause_recode",
"130_infant_cause_recode", "39_cause_recode", "bridged_race_flag", "race_imputation_flag", "race_recode_3", "race_recode_5", "hispanic_origin");
  data = data.filter((data.detail_age_type == 1))
  data = data.drop("detail_age_type")
  data = data.drop("number_of_entity_axis_conditions","entity_condition_4","entity_condition_5","entity_condition_6","entity_condition_7","entity_condition_8",
                                            "entity_condition_9","entity_condition_10","entity_condition_11","entity_condition_12","entity_condition_13","entity_condition_14",
                                            "entity_condition_15","entity_condition_16","entity_condition_17","entity_condition_18","entity_condition_19","entity_condition_20",
                                            "number_of_record_axis_conditions", "record_condition_1", "record_condition_2","record_condition_3","record_condition_4","record_condition_5",
                                            "record_condition_6","record_condition_7","record_condition_8","record_condition_9","record_condition_10","record_condition_11","record_condition_12",
                                            "record_condition_13","record_condition_14","record_condition_15","record_condition_16","record_condition_17","record_condition_18","record_condition_19",
                                            "record_condition_20",)
  data = data.withColumnRenamed("resident_status","resident").withColumnRenamed("education_2003_revision","education").withColumnRenamed("detail_age", "age").withColumnRenamed("current_date_year", "year").withColumnRenamed("icd_code_10th_revision", "icd_death_code").withColumnRenamed("hispanic_originrace_recode", "hispanic")
  data = data.withColumn("resident",col("resident").cast("integer")).withColumn("education",col("education").cast("integer")).withColumn("month_of_death",col("month_of_death").cast("integer")).withColumn("age",col("age").cast("integer")).withColumn("day_of_week_of_death",col("day_of_week_of_death").cast("integer")).withColumn("current_data_year",col("current_data_year").cast("integer")).withColumn("manner_of_death",col("manner_of_death").cast("integer")).withColumn("race",col("race").cast("integer")).withColumn("hispanic",col("hispanic").cast("integer"))
  data = data.fillna(value="0",subset=["entity_condition_1"]).fillna(value="0",subset=["entity_condition_2"]).fillna(value="0",subset=["entity_condition_3"])
  return data


In [21]:
#function for other data sets to be cleaned up - only can be used for 2014 and 2015 and newer, older pages use a different education marker
def data_clean_up2(data):
  data = data.drop("education_2003_revision","education_reporting_flag","age_substitution_flag","age_recode_52","age_recode_27","age_recode_12", "infant_age_recode_22",
"place_of_death_and_decedents_status","method_of_disposition","autopsy", "activity_code", "place_of_injury_for_causes_w00_y34_except_y06_and_y07_", "358_cause_recode", "113_cause_recode",
"130_infant_cause_recode", "39_cause_recode", "bridged_race_flag", "race_imputation_flag", "race_recode_3", "race_recode_5", "hispanic_origin");
  data = data.filter((data.detail_age_type == 1))
  data = data.drop("detail_age_type")
  data = data.drop("number_of_entity_axis_conditions","entity_condition_4","entity_condition_5","entity_condition_6","entity_condition_7","entity_condition_8",
                                            "entity_condition_9","entity_condition_10","entity_condition_11","entity_condition_12","entity_condition_13","entity_condition_14",
                                            "entity_condition_15","entity_condition_16","entity_condition_17","entity_condition_18","entity_condition_19","entity_condition_20",
                                            "number_of_record_axis_conditions", "record_condition_1", "record_condition_2","record_condition_3","record_condition_4","record_condition_5",
                                            "record_condition_6","record_condition_7","record_condition_8","record_condition_9","record_condition_10","record_condition_11","record_condition_12",
                                            "record_condition_13","record_condition_14","record_condition_15","record_condition_16","record_condition_17","record_condition_18","record_condition_19",
                                            "record_condition_20",)
  data = data.withColumnRenamed("resident_status","resident").withColumnRenamed("education_1989_revision","education").withColumnRenamed("detail_age", "age").withColumnRenamed("current_date_year", "year").withColumnRenamed("icd_code_10th_revision", "icd_death_code").withColumnRenamed("hispanic_originrace_recode", "hispanic")
  data = data.withColumn("resident",col("resident").cast("integer")).withColumn("education",col("education").cast("integer")).withColumn("month_of_death",col("month_of_death").cast("integer")).withColumn("age",col("age").cast("integer")).withColumn("day_of_week_of_death",col("day_of_week_of_death").cast("integer")).withColumn("current_data_year",col("current_data_year").cast("integer")).withColumn("manner_of_death",col("manner_of_death").cast("integer")).withColumn("race",col("race").cast("integer")).withColumn("hispanic",col("hispanic").cast("integer"))
  data = data.fillna(value="0",subset=["entity_condition_1"]).fillna(value="0",subset=["entity_condition_2"]).fillna(value="0",subset=["entity_condition_3"])
  return data

In [130]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2014_data.csv"
spark.sparkContext.addFile(url)
death_2014 = spark.read.csv(SparkFiles.get("2014_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2014_final = data_clean_up(death_2014)
death_2014_final.show()

+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|icd_death_code|entity_condition_1|entity_condition_2|entity_condition_3|race|hispanic|
+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|       1|        2|             1|  M| 87|             M|                   4|             2014|             U|              7|           I64|             11I64|                 0|                 0|   1|       6|
|       1|        2|             1|  M| 58|             D|                   3|             2014|             U|              7|          I2

In [140]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2013_data.csv"
spark.sparkContext.addFile(url)
death_2013 = spark.read.csv(SparkFiles.get("2013_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2013_final = data_clean_up(death_2013)
death_2013.show()

+---------------+-----------------------+-----------------------+------------------------+--------------+---+---------------+----------+---------------------+-------------+-------------+-------------+--------------------+-----------------------------------+--------------+--------------------+-----------------+--------------+---------------+---------------------+-------+-------------+------------------------------------------------------+----------------------+----------------+----------------+-----------------------+---------------+--------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------------------+-----

In [132]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2012_data.csv"
spark.sparkContext.addFile(url)
death_2012 = spark.read.csv(SparkFiles.get("2012_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2012_final = data_clean_up(death_2012)
death_2012_final.show()

+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+-----------+------------------+------------------+------------------+----+--------+
|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|icd_code_10|entity_condition_1|entity_condition_2|entity_condition_3|race|hispanic|
+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+-----------+------------------+------------------+------------------+----+--------+
|       1|     null|             1|  F| 59|             W|                   1|             2012|             U|              7|       C349|            11C349|                 0|                 0|   1|       6|
|       1|     null|             1|  F| 90|             W|                   2|             2012|             U|              7|        F03|            

In [133]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2011_data.csv"
spark.sparkContext.addFile(url)
death_2011 = spark.read.csv(SparkFiles.get("2011_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2011_final = data_clean_up(death_2011)
death_2011_final.show()

+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|icd_death_code|entity_condition_1|entity_condition_2|entity_condition_3|race|hispanic|
+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|       2|     null|             1|  F| 65|             M|                   2|             2011|             U|              7|          C859|            11C859|                 0|                 0|   1|       6|
|       1|     null|             1|  F| 57|             S|                   7|             2011|             U|              7|          C5

In [134]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2010_data.csv"
spark.sparkContext.addFile(url)
death_2010 = spark.read.csv(SparkFiles.get("2010_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2010_final = data_clean_up(death_2010)
death_2010_final.show()

+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|icd_death_code|entity_condition_1|entity_condition_2|entity_condition_3|race|hispanic|
+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|       2|     null|             1|  F| 68|             D|                   6|             2010|             U|              7|          G049|            11G049|             61F03|                 0|   3|       8|
|       1|     null|             1|  F| 12|             S|                   1|             2010|             U|              7|           C

In [135]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2009_data.csv"
spark.sparkContext.addFile(url)
death_2009 = spark.read.csv(SparkFiles.get("2009_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2009_final = data_clean_up(death_2009)
death_2009_final.show()

+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|icd_death_code|entity_condition_1|entity_condition_2|entity_condition_3|race|hispanic|
+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|       1|     null|             1|  F| 80|             M|                   2|             2009|             U|              7|          F329|             11E46|            21F329|             61I64|  68|       8|
|       1|     null|             1|  M| 77|             W|                   1|             2009|             U|              7|          C9

In [136]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2008_data.csv"
spark.sparkContext.addFile(url)
death_2008 = spark.read.csv(SparkFiles.get("2008_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2008_final = data_clean_up(death_2008)
death_2008_final.show()

+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|icd_death_code|entity_condition_1|entity_condition_2|entity_condition_3|race|hispanic|
+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|       1|     null|             1|  M| 65|             M|                   1|             2008|             U|              7|          C159|            11C159|                 0|                 0|   1|       6|
|       1|     null|             1|  M| 78|             M|                   7|             2008|             U|              7|          I2

In [137]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2007_data.csv"
spark.sparkContext.addFile(url)
death_2007 = spark.read.csv(SparkFiles.get("2007_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2007_final = data_clean_up(death_2007)
death_2007_final.show()

+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|icd_death_code|entity_condition_1|entity_condition_2|entity_condition_3|race|hispanic|
+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|       3|     null|             1|  M| 72|             M|                   2|             2007|             U|              7|          C259|            11C259|                 0|                 0|   1|       6|
|       1|     null|             1|  M| 35|             M|                   2|             2007|             U|              2|           X

In [138]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2006_data.csv"
spark.sparkContext.addFile(url)
death_2006 = spark.read.csv(SparkFiles.get("2006_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2006_final = data_clean_up(death_2006)
death_2006_final.show()

+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|icd_death_code|entity_condition_1|entity_condition_2|entity_condition_3|race|hispanic|
+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|       1|     null|             1|  F| 84|             W|                   2|             2006|             U|           null|          I500|            11I500|                 0|                 0|   1|       6|
|       1|     null|             1|  F| 93|             W|                   1|             2006|             U|           null|          I2

In [139]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2005_data.csv"
spark.sparkContext.addFile(url)
death_2005 = spark.read.csv(SparkFiles.get("2005_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2005_final = data_clean_up(death_2005)
death_2005_final.show()

+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|icd_death_code|entity_condition_1|entity_condition_2|entity_condition_3|race|hispanic|
+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+
|       1|     null|             1|  F| 45|             M|                   2|             2005|             U|              7|          C439|            11C439|                 0|                 0|   1|       6|
|       1|     null|             1|  M| 61|             D|                   7|             2005|             U|              7|          J4

In [34]:
# from pyspark.sql.window import Window
# from pyspark.sql.functions import rowNumber
# w = Window().orderBy()
# death_2015_final3 = death_2015_final.withColumn("id", rowNumber().over(w))


ModuleNotFoundError: ignored

In [46]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rowNumber

ImportError: ignored

In [48]:
from pyspark.sql.functions import monotonically_increasing_id

death_2015_final2 = death_2015_final.withColumn("id", monotonically_increasing_id() +1)

In [49]:
death_2015_final2.show()

+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+---+
|resident|education|month_of_death|sex|age|marital_status|day_of_week_of_death|current_data_year|injury_at_work|manner_of_death|icd_death_code|entity_condition_1|entity_condition_2|entity_condition_3|race|hispanic| id|
+--------+---------+--------------+---+---+--------------+--------------------+-----------------+--------------+---------------+--------------+------------------+------------------+------------------+----+--------+---+
|       1|        3|             1|  M| 84|             M|                   1|             2015|             U|              7|          I500|            11I500|            61L031|                 0|   1|       6|  1|
|       1|        6|             1|  M| 70|             M|                   2|             2015|             U|            

In [147]:
death_2015_final2.printSchema()

root
 |-- resident: integer (nullable = true)
 |-- education: integer (nullable = true)
 |-- month_of_death: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- day_of_week_of_death: integer (nullable = true)
 |-- current_data_year: integer (nullable = true)
 |-- injury_at_work: string (nullable = true)
 |-- manner_of_death: integer (nullable = true)
 |-- icd_death_code: string (nullable = true)
 |-- entity_condition_1: string (nullable = false)
 |-- entity_condition_2: string (nullable = false)
 |-- entity_condition_3: string (nullable = false)
 |-- race: integer (nullable = true)
 |-- hispanic: integer (nullable = true)
 |-- id: long (nullable = false)



--2021-04-10 19:03:50--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2021-04-10 19:03:50 (5.96 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [52]:
mode = "append"
jdbc_url="jdbc:postgresql://database-1.ck7rneirj52d.us-east-2.rds.amazonaws.com:5432/Death_Database"
config = {"user":"root", 
          "password": "1*fFy4*g1UGy", 
          "driver":"org.postgresql.Driver"}

In [53]:
death_2015_final2.write.jdbc(url=jdbc_url, table='cdc_death_table', mode=mode, properties=config)